## Problem 1 - Data Preparation and Train/Validation/Test Split

**Tasks:**

1. Load the combined water quality and parcel summary data
2. Identify all unique lakes in the dataset
3. Use stratified random sampling to partition lakes into three sets:
   - **Training lakes (50%):** Used to train models
   - **Validation lakes (30%):** Used to evaluate final model performance
   - **Test lakes (20%):** Used for hyperparameter tuning via cross-validation
4. Create feature matrices and target vectors for both prediction targets (Secchi depth and phosphorus)

**Note:** We partition by lakes (not rows) to ensure that all observations from the same lake are in the same set. This prevents data leakage and provides a more realistic evaluation of model generalization.

> <font color="orange"> Your thoughts on the data split strategy here </font>

In [1]:
# Import required libraries
import polars as pl
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
# Load the combined water quality and parcel data
(model_data := pl.read_csv("water_quality_and_parcel_summaries_2004_to_2015.csv"))
# Display shape and first few rows
print(f"Data shape: {model_data.shape}")
model_data.head()

Data shape: (1687, 17)


Monit_MAP_CODE1,LAKE_NAME,Year,phosphorus_avg,phosphorus_count,secchi_avg,secchi_count,mean_FIN_SQ_FT,median_FIN_SQ_FT,mean_acres,median_acres,mean_EMV_TOTAL,median_EMV_TOTAL,mean_total_tax,median_total_tax,pct_garage,pct_basement
str,str,i64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""10004800-01""","""Wasserman Lake""",2006,0.167969,32,1.25,12,1850.365217,1850.365217,1.596775,1.596775,380258.972332,380258.972332,3576.527273,3576.527273,0.664379,null
"""19002500-01""","""Keller Lake""",2004,0.039733,15,1.52,15,2171.247892,2171.247892,0.400093,0.400093,248076.897133,248076.897133,2338.893339,2338.893339,null,null
"""82007600-01""","""Barker Lake""",2008,0.108571,7,1.197429,7,0.0,0.0,22.651304,22.651304,352650.931677,352650.931677,263408.695652,263408.695652,null,null
"""82010900-01""","""Eagle Point Lake""",2007,0.1705,2,0.85,2,0.0,0.0,0.0,0.0,875000.0,875000.0,0.0,0.0,null,null
"""02000500-01""","""George Watch Lake""",2006,0.164286,7,0.728571,7,1637.532154,1637.532154,2.23492,2.23492,233350.482315,233350.482315,3405.800643,3405.800643,null,0.918143


In [4]:
# Examine the target variables
model_data.select(["secchi_avg", "phosphorus_avg", "Monit_MAP_CODE1", "LAKE_NAME", "Year"]).head(10)

secchi_avg,phosphorus_avg,Monit_MAP_CODE1,LAKE_NAME,Year
f64,f64,str,str,i64
1.25,0.167969,"""10004800-01""","""Wasserman Lake""",2006
1.52,0.039733,"""19002500-01""","""Keller Lake""",2004
1.197429,0.108571,"""82007600-01""","""Barker Lake""",2008
0.85,0.1705,"""82010900-01""","""Eagle Point Lake""",2007
0.728571,0.164286,"""02000500-01""","""George Watch Lake""",2006
0.662143,0.113143,"""19009500-01""","""Seidl Lake""",2006
2.991667,0.022,"""82010300-01""","""Olson Lake""",2012
1.456364,0.034591,"""13005300-01""","""Big Comfort Lake""",2014
1.94,0.04,"""19002700-01""","""Crystal Lake""",2004


In [5]:
# Extract unique lakes and create the train/validation/test split
(lakes := model_data.select("Monit_MAP_CODE1")
                    .unique()
                    .to_series()
                    .to_list())

print(f"Total unique lakes: {len(lakes)}")

Total unique lakes: 263


In [6]:
# Set random seed for reproducibility
np.random.seed(123)

# Create validation set (30% of lakes)
validation_ids = np.random.choice(
    lakes,
    size=int(0.30 * len(lakes)),
    replace=False
)

# Create training and test sets from remaining lakes
remaining = np.setdiff1d(lakes, validation_ids)

train_ids = np.random.choice(
    remaining,
    size=int(0.70 * len(remaining)),
    replace=False
)

test_ids = np.setdiff1d(remaining, train_ids)

print(f"Training lakes: {len(train_ids)}")
print(f"Validation lakes: {len(validation_ids)}")
print(f"Test lakes: {len(test_ids)}")

Training lakes: 129
Validation lakes: 78
Test lakes: 56


In [7]:
# Add a column to indicate the split (training/validation/test)
(model_data_2 := model_data.with_columns(
    pl.when(pl.col("Monit_MAP_CODE1").is_in(validation_ids))
      .then(pl.lit("Validation"))
      .when(pl.col("Monit_MAP_CODE1").is_in(train_ids))
      .then(pl.lit("Training"))
      .otherwise(pl.lit("Test"))
      .alias("TrainingValidation")
))

Monit_MAP_CODE1,LAKE_NAME,Year,phosphorus_avg,phosphorus_count,secchi_avg,secchi_count,mean_FIN_SQ_FT,median_FIN_SQ_FT,mean_acres,median_acres,mean_EMV_TOTAL,median_EMV_TOTAL,mean_total_tax,median_total_tax,pct_garage,pct_basement,TrainingValidation
str,str,i64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""10004800-01""","""Wasserman Lake""",2006,0.167969,32,1.25,12,1850.365217,1850.365217,1.596775,1.596775,380258.972332,380258.972332,3576.527273,3576.527273,0.664379,null,"""Test"""
"""19002500-01""","""Keller Lake""",2004,0.039733,15,1.52,15,2171.247892,2171.247892,0.400093,0.400093,248076.897133,248076.897133,2338.893339,2338.893339,null,null,"""Validation"""
"""82007600-01""","""Barker Lake""",2008,0.108571,7,1.197429,7,0.0,0.0,22.651304,22.651304,352650.931677,352650.931677,263408.695652,263408.695652,null,null,"""Validation"""
"""82010900-01""","""Eagle Point Lake""",2007,0.1705,2,0.85,2,0.0,0.0,0.0,0.0,875000.0,875000.0,0.0,0.0,null,null,"""Training"""
"""02000500-01""","""George Watch Lake""",2006,0.164286,7,0.728571,7,1637.532154,1637.532154,2.23492,2.23492,233350.482315,233350.482315,3405.800643,3405.800643,null,0.918143,"""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""02065400-01""","""Cenaiko Lake""",2008,0.014615,13,1.876923,13,673.400151,673.400151,0.891502,0.891502,251546.816127,251546.816127,3189.853806,3189.853806,null,0.862924,"""Validation"""
"""82008000-01""","""Halfbreed Lake""",2011,0.021455,11,4.254545,11,1003.516199,1003.516199,3.105356,3.105356,194758.099352,194758.099352,0.0,0.0,1.0,1.0,"""Training"""
"""27010400-01""","""Medicine Lake""",2011,0.04496,25,1.74,25,865.938228,865.938228,0.701334,0.701334,333803.088578,333803.088578,6511.749417,6511.749417,1.0,1.0,"""Test"""


In [8]:
# Create separate dataframes for each set
training_data = model_data_2.filter(pl.col("TrainingValidation") == "Training")
validation_data = model_data_2.filter(pl.col("TrainingValidation") == "Validation")
test_data = model_data_2.filter(pl.col("TrainingValidation") == "Test")

# Display row counts
print(f"Training rows: {training_data.shape[0]}")
print(f"Validation rows: {validation_data.shape[0]}")
print(f"Test rows: {test_data.shape[0]}")

Training rows: 842
Validation rows: 501
Test rows: 344


In [9]:
# Check the distribution
model_data_2.select(pl.col("TrainingValidation").value_counts())

TrainingValidation
struct[2]
"{""Validation"",501}"
"{""Test"",344}"
"{""Training"",842}"


In [10]:
# Define columns to drop (non-features and target variables)
drop_cols = {
    "secchi_avg",
    "phosphorus_avg",
    "secchi_count",
    "phosphorus_count",
    "TrainingValidation",
    "Monit_MAP_CODE1",
    "LAKE_NAME",
    "Year"
}

## Problem 2 - Model Training and Comparison

**Tasks:**

1. Prepare feature matrices (X) and target vectors (y) for both prediction targets
2. Set up hyperparameter grids for Decision Tree and Random Forest models
3. Use GridSearchCV with 10-fold cross-validation on the training set to find optimal hyperparameters
4. Compare model performance on the test set
5. Select the best model for each target variable

### Target 1: Average Secchi Depth

In [11]:
# Prepare features and targets for Secchi Depth prediction
X_train = training_data.drop(drop_cols).to_pandas()
y_train = training_data.select("secchi_avg").to_pandas()

X_val = validation_data.drop(drop_cols).to_pandas()
y_val = validation_data.select("secchi_avg").to_pandas()

X_test = test_data.drop(drop_cols).to_pandas()
y_test = test_data.select("secchi_avg").to_pandas()

print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")
print(f"Test set size: {X_test.shape}")

Training set size: (842, 10)
Validation set size: (501, 10)
Test set size: (344, 10)


In [17]:
# Check for and remove NaN values
print(f"Before removing NaN:")
print(f"  X_train NaN count: {X_train.isna().sum().sum()}")
print(f"  y_train NaN count: {y_train.isna().sum().sum()}")

# Remove rows with NaN values in target or features
valid_train = ~(X_train.isna().any(axis=1) | y_train.isna().any(axis=1))
X_train = X_train[valid_train].reset_index(drop=True)
y_train = y_train[valid_train].reset_index(drop=True)

valid_val = ~(X_val.isna().any(axis=1) | y_val.isna().any(axis=1))
X_val = X_val[valid_val].reset_index(drop=True)
y_val = y_val[valid_val].reset_index(drop=True)

valid_test = ~(X_test.isna().any(axis=1) | y_test.isna().any(axis=1))
X_test = X_test[valid_test].reset_index(drop=True)
y_test = y_test[valid_test].reset_index(drop=True)

print(f"\nAfter removing NaN:")
print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")
print(f"Test set size: {X_test.shape}")

Before removing NaN:
  X_train NaN count: 1090
  y_train NaN count: 13

After removing NaN:
Training set size: (272, 10)
Validation set size: (173, 10)
Test set size: (107, 10)


In [12]:
# Define hyperparameter grids for model tuning
# Decision Tree hyperparameters
dt_param_grid = {
    'max_depth': [None, 1, 2, 3, 4, 5, 7, 10],
    'min_samples_split': [2, 3, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

# Random Forest hyperparameters
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 3, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['sqrt', 'log2']
}

In [18]:
# Set up cross-validation
state = 45854
cv_obj = KFold(n_splits=10, shuffle=True, random_state=state)
cv_obj

KFold(n_splits=10, random_state=45854, shuffle=True)

In [19]:
# Initialize GridSearchCV for Decision Tree
grid_search_dt = GridSearchCV(
    DecisionTreeRegressor(random_state=state),
    dt_param_grid,
    cv=cv_obj,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

In [20]:
# Initialize GridSearchCV for Random Forest
grid_search_rf = GridSearchCV(
    RandomForestRegressor(random_state=state),
    rf_param_grid,
    cv=cv_obj,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

In [21]:
# Train Decision Tree model
print("Training Decision Tree...")
grid_search_dt.fit(X_train, y_train.values.ravel())

Training Decision Tree...
Fitting 10 folds for each of 96 candidates, totalling 960 fits


,estimator,DecisionTreeR...m_state=45854)
,param_grid,"{'max_depth': [None, 1, ...], 'min_samples_leaf': [1, 5, ...], 'min_samples_split': [2, 3, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [22]:
# Display Decision Tree results
print(f"Best CV Score: {grid_search_dt.best_score_:.4f}")
print(f"Best Parameters: {grid_search_dt.best_params_}")
(best_dt := grid_search_dt.best_estimator_)

Best CV Score: 0.3931
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 3}


,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,3
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,45854
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [23]:
# Train Random Forest model
print("Training Random Forest...")
grid_search_rf.fit(X_train, y_train.values.ravel())

Training Random Forest...
Fitting 10 folds for each of 288 candidates, totalling 2880 fits


,estimator,RandomForestR...m_state=45854)
,param_grid,"{'max_depth': [5, 10, ...], 'max_features': ['sqrt', 'log2'], 'min_samples_leaf': [1, 5, ...], 'min_samples_split': [2, 3, ...], ...}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,100


In [24]:
# Display Random Forest results
print(f"Best CV Score: {grid_search_rf.best_score_:.4f}")
print(f"Best Parameters: {grid_search_rf.best_params_}")
(best_rf := grid_search_rf.best_estimator_)

Best CV Score: 0.6092
Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


### Evaluating Models on Test Set - Secchi Depth

In [25]:
# Make predictions with Decision Tree
y_test_pred_dt = best_dt.predict(X_test)

# Evaluate Decision Tree
dt_mse = mean_squared_error(y_test, y_test_pred_dt)
dt_rmse = np.sqrt(dt_mse)
dt_r2 = r2_score(y_test, y_test_pred_dt)
dt_mae = mean_absolute_error(y_test, y_test_pred_dt)

print("Decision Tree Performance (Test Set):")
print(f"  R² Score: {dt_r2:.4f}")
print(f"  RMSE: {dt_rmse:.4f}")
print(f"  MAE: {dt_mae:.4f}")

Decision Tree Performance (Test Set):
  R² Score: -0.8683
  RMSE: 1.7720
  MAE: 1.3594


In [26]:
# Make predictions with Random Forest
y_test_pred_rf = best_rf.predict(X_test)

# Evaluate Random Forest
rf_mse = mean_squared_error(y_test, y_test_pred_rf)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(y_test, y_test_pred_rf)
rf_mae = mean_absolute_error(y_test, y_test_pred_rf)

print("Random Forest Performance (Test Set):")
print(f"  R² Score: {rf_r2:.4f}")
print(f"  RMSE: {rf_rmse:.4f}")
print(f"  MAE: {rf_mae:.4f}")

Random Forest Performance (Test Set):
  R² Score: -0.3141
  RMSE: 1.4861
  MAE: 1.0939


In [27]:
# Compare models
print("\nModel Comparison:")
if dt_r2 > rf_r2:
    print("Decision Tree performs better")
    best_model_secchi = best_dt
    best_model_name = "Decision Tree"
else:
    print("Random Forest performs better")
    best_model_secchi = best_rf
    best_model_name = "Random Forest"


Model Comparison:
Random Forest performs better


### Target 2: Average Total Phosphorus

In [28]:
# Prepare features and targets for Phosphorus prediction
X_train_tp = training_data.drop(drop_cols).to_pandas()
y_train_tp = training_data.select("phosphorus_avg").to_pandas()

X_val_tp = validation_data.drop(drop_cols).to_pandas()
y_val_tp = validation_data.select("phosphorus_avg").to_pandas()

X_test_tp = test_data.drop(drop_cols).to_pandas()
y_test_tp = test_data.select("phosphorus_avg").to_pandas()

print(f"Training set size: {X_train_tp.shape}")
print(f"Validation set size: {X_val_tp.shape}")
print(f"Test set size: {X_test_tp.shape}")

Training set size: (842, 10)
Validation set size: (501, 10)
Test set size: (344, 10)


In [29]:
# Check for and remove NaN values for Phosphorus data
print(f"Before removing NaN:")
print(f"  X_train_tp NaN count: {X_train_tp.isna().sum().sum()}")
print(f"  y_train_tp NaN count: {y_train_tp.isna().sum().sum()}")

# Remove rows with NaN values in target or features
valid_train_tp = ~(X_train_tp.isna().any(axis=1) | y_train_tp.isna().any(axis=1))
X_train_tp = X_train_tp[valid_train_tp].reset_index(drop=True)
y_train_tp = y_train_tp[valid_train_tp].reset_index(drop=True)

valid_val_tp = ~(X_val_tp.isna().any(axis=1) | y_val_tp.isna().any(axis=1))
X_val_tp = X_val_tp[valid_val_tp].reset_index(drop=True)
y_val_tp = y_val_tp[valid_val_tp].reset_index(drop=True)

valid_test_tp = ~(X_test_tp.isna().any(axis=1) | y_test_tp.isna().any(axis=1))
X_test_tp = X_test_tp[valid_test_tp].reset_index(drop=True)
y_test_tp = y_test_tp[valid_test_tp].reset_index(drop=True)

print(f"\nAfter removing NaN:")
print(f"Training set size: {X_train_tp.shape}")
print(f"Validation set size: {X_val_tp.shape}")
print(f"Test set size: {X_test_tp.shape}")

Before removing NaN:
  X_train_tp NaN count: 1090
  y_train_tp NaN count: 0

After removing NaN:
Training set size: (276, 10)
Validation set size: (180, 10)
Test set size: (112, 10)


In [30]:
# Initialize and train Decision Tree for Phosphorus
grid_search_dt_tp = GridSearchCV(
    DecisionTreeRegressor(random_state=state),
    dt_param_grid,
    cv=cv_obj,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

print("Training Decision Tree for Phosphorus...")
grid_search_dt_tp.fit(X_train_tp, y_train_tp.values.ravel())

Training Decision Tree for Phosphorus...
Fitting 10 folds for each of 96 candidates, totalling 960 fits


,estimator,DecisionTreeR...m_state=45854)
,param_grid,"{'max_depth': [None, 1, ...], 'min_samples_leaf': [1, 5, ...], 'min_samples_split': [2, 3, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [31]:
# Display Decision Tree results for Phosphorus
print(f"Best CV Score: {grid_search_dt_tp.best_score_:.4f}")
print(f"Best Parameters: {grid_search_dt_tp.best_params_}")
(best_dt_tp := grid_search_dt_tp.best_estimator_)

Best CV Score: 0.1441
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}


,criterion,'squared_error'
,splitter,'best'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,45854
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [32]:
# Initialize and train Random Forest for Phosphorus
grid_search_rf_tp = GridSearchCV(
    RandomForestRegressor(random_state=state),
    rf_param_grid,
    cv=cv_obj,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

print("Training Random Forest for Phosphorus...")
grid_search_rf_tp.fit(X_train_tp, y_train_tp.values.ravel())

Training Random Forest for Phosphorus...
Fitting 10 folds for each of 288 candidates, totalling 2880 fits


,estimator,RandomForestR...m_state=45854)
,param_grid,"{'max_depth': [5, 10, ...], 'max_features': ['sqrt', 'log2'], 'min_samples_leaf': [1, 5, ...], 'min_samples_split': [2, 3, ...], ...}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,300


In [33]:
# Display Random Forest results for Phosphorus
print(f"Best CV Score: {grid_search_rf_tp.best_score_:.4f}")
print(f"Best Parameters: {grid_search_rf_tp.best_params_}")
(best_rf_tp := grid_search_rf_tp.best_estimator_)

Best CV Score: 0.3361
Best Parameters: {'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


,n_estimators,300
,criterion,'squared_error'
,max_depth,15
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


### Evaluating Models on Test Set - Phosphorus

In [34]:
# Make predictions with Decision Tree for Phosphorus
y_test_pred_dt_tp = best_dt_tp.predict(X_test_tp)

# Evaluate Decision Tree
dt_mse_tp = mean_squared_error(y_test_tp, y_test_pred_dt_tp)
dt_rmse_tp = np.sqrt(dt_mse_tp)
dt_r2_tp = r2_score(y_test_tp, y_test_pred_dt_tp)
dt_mae_tp = mean_absolute_error(y_test_tp, y_test_pred_dt_tp)

print("Decision Tree Performance (Test Set):")
print(f"  R² Score: {dt_r2_tp:.4f}")
print(f"  RMSE: {dt_rmse_tp:.4f}")
print(f"  MAE: {dt_mae_tp:.4f}")

Decision Tree Performance (Test Set):
  R² Score: -0.3705
  RMSE: 0.1784
  MAE: 0.0941


In [35]:
# Make predictions with Random Forest for Phosphorus
y_test_pred_rf_tp = best_rf_tp.predict(X_test_tp)

# Evaluate Random Forest
rf_mse_tp = mean_squared_error(y_test_tp, y_test_pred_rf_tp)
rf_rmse_tp = np.sqrt(rf_mse_tp)
rf_r2_tp = r2_score(y_test_tp, y_test_pred_rf_tp)
rf_mae_tp = mean_absolute_error(y_test_tp, y_test_pred_rf_tp)

print("Random Forest Performance (Test Set):")
print(f"  R² Score: {rf_r2_tp:.4f}")
print(f"  RMSE: {rf_rmse_tp:.4f}")
print(f"  MAE: {rf_mae_tp:.4f}")

Random Forest Performance (Test Set):
  R² Score: -0.0930
  RMSE: 0.1593
  MAE: 0.0764


In [36]:
# Compare models
print("\nModel Comparison:")
if dt_r2_tp > rf_r2_tp:
    print("Decision Tree performs better")
    best_model_phosphorus = best_dt_tp
    best_model_name_tp = "Decision Tree"
else:
    print("Random Forest performs better")
    best_model_phosphorus = best_rf_tp
    best_model_name_tp = "Random Forest"


Model Comparison:
Random Forest performs better


## Problem 3 - Final Model Retraining and Validation

**Tasks:**

1. Combine the training and test datasets
2. Retrain the best model for each target on the combined dataset
3. Evaluate the final model performance on the held-out validation lakes
4. Compare final performance to the test set performance

### Retraining for Secchi Depth

In [37]:
# Combine training and test data for final retraining
final_train = training_data.vstack(test_data)
final_X = final_train.drop(drop_cols).to_pandas()
final_y = final_train.select("secchi_avg").to_pandas()

print(f"Final training set size: {final_X.shape}")

Final training set size: (1186, 10)


In [39]:
# Remove NaN values from combined final training data
print(f"Before removing NaN:")
print(f"  final_X NaN count: {final_X.isna().sum().sum()}")
print(f"  final_y NaN count: {final_y.isna().sum().sum()}")

# Remove rows with NaN values in target or features
valid_final = ~(final_X.isna().any(axis=1) | final_y.isna().any(axis=1))
final_X = final_X[valid_final].reset_index(drop=True)
final_y = final_y[valid_final].reset_index(drop=True)

print(f"\nAfter removing NaN:")
print(f"Final training set size: {final_X.shape}")

Before removing NaN:
  final_X NaN count: 1489
  final_y NaN count: 24

After removing NaN:
Final training set size: (379, 10)


In [40]:
# Retrain the best model on combined data
best_model_secchi_final = best_model_secchi.fit(final_X, final_y.values.ravel())
print(f"Retrained {best_model_name} model")

Retrained Random Forest model


In [41]:
# Evaluate on validation set
y_val_pred_secchi = best_model_secchi_final.predict(X_val)

val_r2_secchi = r2_score(y_val, y_val_pred_secchi)
val_rmse_secchi = np.sqrt(mean_squared_error(y_val, y_val_pred_secchi))
val_mae_secchi = mean_absolute_error(y_val, y_val_pred_secchi)

print("\nFinal Model Performance on Validation Set (Secchi Depth):")
print(f"  R² Score: {val_r2_secchi:.4f}")
print(f"  RMSE: {val_rmse_secchi:.4f}")
print(f"  MAE: {val_mae_secchi:.4f}")


Final Model Performance on Validation Set (Secchi Depth):
  R² Score: -0.2104
  RMSE: 1.3772
  MAE: 0.9943


### Retraining for Phosphorus

In [42]:
# Prepare combined training data for Phosphorus
final_X_tp = final_train.drop(drop_cols).to_pandas()
final_y_tp = final_train.select("phosphorus_avg").to_pandas()

print(f"Final training set size: {final_X_tp.shape}")

Final training set size: (1186, 10)


In [43]:
# Remove NaN values from combined final training data for Phosphorus
print(f"Before removing NaN:")
print(f"  final_X_tp NaN count: {final_X_tp.isna().sum().sum()}")
print(f"  final_y_tp NaN count: {final_y_tp.isna().sum().sum()}")

# Remove rows with NaN values in target or features
valid_final_tp = ~(final_X_tp.isna().any(axis=1) | final_y_tp.isna().any(axis=1))
final_X_tp = final_X_tp[valid_final_tp].reset_index(drop=True)
final_y_tp = final_y_tp[valid_final_tp].reset_index(drop=True)

print(f"\nAfter removing NaN:")
print(f"Final training set size: {final_X_tp.shape}")

Before removing NaN:
  final_X_tp NaN count: 1489
  final_y_tp NaN count: 0

After removing NaN:
Final training set size: (388, 10)


In [44]:
# Retrain the best model on combined data
best_model_phosphorus_final = best_model_phosphorus.fit(final_X_tp, final_y_tp.values.ravel())
print(f"Retrained {best_model_name_tp} model")

Retrained Random Forest model


In [45]:
# Evaluate on validation set
y_val_pred_phosphorus = best_model_phosphorus_final.predict(X_val_tp)

val_r2_phosphorus = r2_score(y_val_tp, y_val_pred_phosphorus)
val_rmse_phosphorus = np.sqrt(mean_squared_error(y_val_tp, y_val_pred_phosphorus))
val_mae_phosphorus = mean_absolute_error(y_val_tp, y_val_pred_phosphorus)

print("\nFinal Model Performance on Validation Set (Phosphorus):")
print(f"  R² Score: {val_r2_phosphorus:.4f}")
print(f"  RMSE: {val_rmse_phosphorus:.4f}")
print(f"  MAE: {val_mae_phosphorus:.4f}")


Final Model Performance on Validation Set (Phosphorus):
  R² Score: -0.1557
  RMSE: 0.0930
  MAE: 0.0646


## Summary and Conclusions

**Summary of Results:**

- For Secchi Depth: {best_model_name} was selected as the best model with a validation R² of {val_r2_secchi:.4f}
- For Phosphorus: {best_model_name_tp} was selected as the best model with a validation R² of {val_r2_phosphorus:.4f}
